In [1]:
import numpy as np
import os
import tensorflow as tf
import sklearn.metrics as sk
from sklearn.model_selection import KFold

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

###### Do not modify here ###### 

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


Model functions

In [ ]:
#transform to one_hot array
def one_hot(input_data):
    one_hot = []
    for item in input_data:
        if item == 0.:
            one_h = [1.,0.,0.,0.,0.]
        elif item == 1.:
            one_h = [0.,1.,0.,0.,0.]
        elif item == 2.:
            one_h = [0.,0.,1.,0.,0.]
        elif item == 3.:
            one_h = [0.,0.,0.,1.,0.]
        elif item == 4.:
            one_h = [0.,0.,0.,0.,1.]

        one_hot.append(one_h)
    one_hot = np.array(one_hot)
    #one_hot = one_hot.reshape(len(one_hot),10,1)
    #one_hot = one_hot.reshape(len(one_hot), 7,1)
    #return tf.constant([one_hot])
    return one_hot

Training and cross validation step

In [ ]:
#define model
INPUT_NODE = 784
NEURONS = 128
OUTPUT_NODE = 5

#define loss (cost) function
true_labels = tf.placeholder(tf.int32, [None,5], name='TL')
x = tf.placeholder(tf.float32, [None,INPUT_NODE], name='X')
dropout_keep_prob = tf.placeholder(tf.float32, name='dropout')

initializer=tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_AVG', uniform=True)

#W_fc1 = tf.Variable(initializer([INPUT_NODE, NEURONS]), name='WLayer1')  # Weights
#b_fc1 = tf.Variable(initializer([NEURONS]), name='bLayer1')  # bias
#h_fc1 = tf.nn.elu(tf.matmul(x, W_fc1) + b_fc1, name='Layer1') # y = Wx + b

h_fc1 = tf.layers.dense(inputs=x, units=NEURONS, kernel_initializer=initializer,activation=tf.nn.relu, name='Layer1')

#W_fc2 = tf.Variable(initializer([NEURONS, NEURONS]), name='WLayer2')  # Weights
#b_fc2 = tf.Variable(initializer([NEURONS]), name='bLayer2')  # bias
#h_fc2 = tf.nn.elu(tf.matmul(h_fc1, W_fc2) + b_fc2, name='Layer2') # y = Wx + b

h_fc2 = tf.layers.dense(inputs=h_fc1, units=NEURONS, kernel_initializer=initializer,activation=tf.nn.relu, name='Layer2')

W_fc3 = tf.Variable(initializer([NEURONS, NEURONS]), name='WLayer3')  # Weights
b_fc3 = tf.Variable(initializer([NEURONS]), name='bLayer3')  # bias
h_fc3 = tf.nn.elu(tf.matmul(h_fc2, W_fc3) + b_fc3, name='Layer3') # y = Wx + b

W_fc4 = tf.Variable(initializer([NEURONS, NEURONS]), name='WLayer4')  # Weights
b_fc4 = tf.Variable(initializer([NEURONS]), name='bLayer4')  # bias
h_fc4 = tf.nn.elu(tf.matmul(h_fc3, W_fc4) + b_fc4, name='Layer4') # y = Wx + b

W_fc5 = tf.Variable(initializer([NEURONS, NEURONS]), name='WLayer5')  # Weights
b_fc5 = tf.Variable(initializer([NEURONS]), name='bLayer5')  # bias
h_fc5 = tf.nn.elu(tf.matmul(h_fc4, W_fc5) + b_fc5, name='Layer5') # y = Wx + b

output = tf.layers.dense(inputs=h_fc5, units=OUTPUT_NODE, kernel_initializer=initializer, name='output')
output_after_softmax = tf.nn.softmax(output, name='output_after_softmax')

#better take unsoftmax output as logits as our cross entropy loss function will do softmax
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits= output,labels= tf.argmax(true_labels, 1)), name='loss') 


train_step = tf.train.AdamOptimizer(0.01).minimize(loss)
tf.add_to_collection("optimizer", train_step)

correct_prediction = tf.equal(tf.argmax(output_after_softmax, 1), tf.argmax(true_labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

# define training parameters
training_epochs = 15
batch_size = 500
train_data_size = np.prod(X_train1.shape[0]) #28038
total_batch = int(train_data_size/batch_size)

sess2 = tf.InteractiveSession()
sess2.run(tf.global_variables_initializer())
saver2 = tf.train.Saver()

# Training cycle
current_best = 0.0
stopping_step = 0
TP_0 = 0
FP_0 = 0
FN_0 = 0
TP_1 = 0
FP_1 = 0
FN_1 = 0
TP_2 = 0
FP_2 = 0
FN_2 = 0
TP_3 = 0
FP_3 = 0
FN_3 = 0
TP_4 = 0
FP_4 = 0
FN_4 = 0
precision_0 = 0.0
recall_0 = 0.0
precision_1 = 0.0
recall_1 = 0.0
precision_2 = 0.0
recall_2 = 0.0
precision_3 = 0.0
recall_3 = 0.0
precision_4 = 0.0
recall_4 = 0.0
epoch_number = 0;
y_train1 =  one_hot(y_train1)
y_test1 =  one_hot(y_test1)
for epoch in range(training_epochs):
    #print ('epoch %f in %f'% (epoch+1, training_epochs))
    epoch_number+=1
    # generate random indexes
    indexes = np.random.permutation(train_data_size)


    # run all datas for one epoch
    for position in range(0, train_data_size, batch_size):
        # generate mini batch ids
        ids = indexes[position:(position+batch_size) if (position+batch_size) < train_data_size else train_data_size]
        batch_xs = X_train1[ids]
        batch_ts = y_train1[ids]

        sess2.run(train_step,feed_dict={x:batch_xs, true_labels:batch_ts, dropout_keep_prob: 1})

    loss_val, acc_val = sess2.run([loss, accuracy],feed_dict={x:X_test1, true_labels:y_test1, dropout_keep_prob: 1})
    predicted_result = sess2.run(output_after_softmax,feed_dict={x:X_test1, true_labels:y_test1, dropout_keep_prob: 1})

    fianl_predict = np.argmax(predicted_result,1)
    labels = np.argmax(y_test1,1)
    
    if(acc_val > current_best):
        current_best = acc_val

    TP_0 = 0
    FP_0 = 0
    FN_0 = 0
    TP_1 = 0
    FP_1 = 0
    FN_1 = 0
    TP_2 = 0
    FP_2 = 0
    FN_2 = 0
    TP_3 = 0
    FP_3 = 0
    FN_3 = 0
    TP_4 = 0
    FP_4 = 0
    FN_4 = 0


    for i in range(0,fianl_predict.size,1):
        if(fianl_predict[i] == 0):
            if(labels[i] == 0):
                TP_0+=1
            else:
                FP_0+=1
        if(fianl_predict[i] == 1):
            if(labels[i] == 1):
                TP_1+=1
            else:
                FP_1+=1
        if(fianl_predict[i] == 2):
            if(labels[i] == 2):
                TP_2+=1
            else:
                FP_2+=1
        if(fianl_predict[i] == 3):
            if(labels[i] == 3):
                TP_3+=1
            else:
                FP_3+=1
        if(fianl_predict[i] == 4):
            if(labels[i] == 4):
                TP_4+=1
            else:
                FP_4+=1
        if(labels[i] == 0):
            if(fianl_predict[i] != 0):
                FN_0+=1
        if (labels[i] == 1):
            if(fianl_predict[i] != 1):
                FN_1+=1
        if (labels[i] == 2):
            if(fianl_predict[i] != 2):
                FN_2+=1
        if (labels[i] == 3):
            if(fianl_predict[i] != 3):
                FN_3+=1
        if (labels[i] == 4):
            if(fianl_predict[i] != 4):
                FN_4+=1

precision_0 = TP_0/(TP_0 + FP_0)
recall_0 = TP_0/(TP_0 + FN_0)
precision_1 = TP_1/(TP_1 + FP_1)
recall_1 = TP_1/(TP_1 + FN_1)
precision_2 = TP_2/(TP_2 + FP_2)
recall_2 = TP_2/(TP_2 + FN_2)
precision_3 = TP_3/(TP_3 + FP_3)
recall_3 = TP_3/(TP_3 + FN_3)
precision_4 = TP_4/(TP_4 + FP_4)
recall_4 = TP_4/(TP_4 + FN_4)
print ('Test accuracy: %f'% (current_best))
print ('Label 0 Precision: %f, Recall: %f'%(precision_0,recall_0))
print ('Label 1 Precision: %f, Recall: %f'%(precision_1,recall_1))
print ('Label 2 Precision: %f, Recall: %f'%(precision_2,recall_2))
print ('Label 3 Precision: %f, Recall: %f'%(precision_3,recall_3))
print ('Label 4 Precision: %f, Recall: %f'%(precision_4,recall_4))
saver2.save(sess2, './checkpointFile/my_test_model')
sess2.close()

Test accuracy: 0.991633
Label 0 Precision: 0.993871, Recall: 0.992857
Label 1 Precision: 0.999116, Recall: 0.995595
Label 2 Precision: 0.988911, Recall: 0.950581
Label 3 Precision: 0.951796, Recall: 0.997030
Label 4 Precision: 0.992850, Recall: 0.989817


Testing step